#### Вариант 18(6)
#### ('alt.atheism', 'sci.space', 'soc.religion.christian')
#### DT, KNN, LR

In [1]:
import warnings
from sklearn.datasets import fetch_20newsgroups
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
categories = ['alt.atheism', 'sci.space', 'soc.religion.christian']
remove = ('headers', 'footers', 'quotes')

twenty_train_full = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, remove=remove)
twenty_test_full = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, remove=remove)

#### Стемминг

In [3]:
import nltk
from nltk import word_tokenize
from nltk.stem import *

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nicolay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def stemming(data):
    porter_stemmer = PorterStemmer()
    stem = []
    for text in data:
        nltk_tokens = word_tokenize(text)
        line = ''.join([' ' + porter_stemmer.stem(word) for word in nltk_tokens])
        stem.append(line)
    return stem

In [5]:
stem_train = stemming(twenty_train_full.data)
stem_test = stemming(twenty_test_full.data)

##DT, KNN, LR

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [7]:
stop_words = [None, 'english']
max_features_values = [100, 500, 1000, 5000, 10000]
use_idf = [True, False]

In [8]:
rf_first = range(1, 5, 1)
rf_second = range(5, 100, 20)

rf_tree_max_depth = [*rf_first, *rf_second]

In [9]:
dt_parameters = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__criterion': ('gini', 'entropy'),
    'clf__max_depth': [*range(1, 6, 1), *range(25, 101, 20)],
}

knn_parameters = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__n_neighbors': [3, 5, 7, 9],
    'clf__metric': ['euclidean', 'manhattan'],
}

lr_parameters = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__solver': ['newton-cg', 'lbfgs', 'sag', 'liblinear'],
    'clf__penalty': ['l2'],
}


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

### Дерево решений (DT):
• критерий (параметр criterion: ‘gini’, ‘entropy’),
• глубина дерева (параметр max_depth от 1 до 5 с шагом 1, далее до 100 с
шагом 20).

#### Без использования стемминга

In [11]:
text_clf_dt = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', DecisionTreeClassifier())])
gscv_dt = GridSearchCV(text_clf_dt, param_grid=dt_parameters, n_jobs=-1)
gscv_dt.fit(twenty_train_full.data, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': [1, 2, 3, 4, 5, 25, 45, 65, 85],
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

#### С использованием стемминга

In [12]:
text_clf_dt_stem = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', DecisionTreeClassifier())])
gscv_dt_stem = GridSearchCV(text_clf_dt_stem, param_grid=dt_parameters, n_jobs=-1)
gscv_dt_stem.fit(stem_train, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': [1, 2, 3, 4, 5, 25, 45, 65, 85],
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

### К-ближайших соседей (KNN):
• количество ближайших соседей,
• метрика (евклидова, городских кварталов) 

#### Без использования стэмминга

In [13]:
text_clf_knn = Pipeline([('vect', CountVectorizer()),
                          ('tfidf', TfidfTransformer()),
                          ('clf', KNeighborsClassifier())])
gscv_knn = GridSearchCV(text_clf_knn, param_grid=knn_parameters, n_jobs=-1)
gscv_knn.fit(twenty_train_full.data, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'clf__metric': ['euclidean', 'manhattan'],
                         'clf__n_neighbors': [3, 5, 7, 9],
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

#### С использованием стэмминга

In [ ]:
text_clf_knn_stem = Pipeline([('vect', CountVectorizer()),
                               ('tfidf', TfidfTransformer()),
                               ('clf', KNeighborsClassifier())])
gscv_knn_stem = GridSearchCV(text_clf_knn_stem, param_grid=knn_parameters, n_jobs=-1)
gscv_knn_stem.fit(stem_train, twenty_train_full.target)

### Логистическая регрессия (LR):
• метод нахождения экстремума (параметр solver: ‘newton-cg’, ‘lbfgs’,
‘sag’, ‘liblinear’),
• регуляризация (параметр penalty: ‘L1’, ‘L2’)

#### Без использования стемминга

In [ ]:
text_clf_lr = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', LogisticRegression())])
gscv_lr = GridSearchCV(text_clf_lr, param_grid=lr_parameters, n_jobs=-1)
gscv_lr.fit(twenty_train_full.data, twenty_train_full.target)


#### С использованием стемминга

In [ ]:
text_clf_lr_stem = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', LogisticRegression())])
gscv_lr_stem = GridSearchCV(text_clf_lr_stem, param_grid=lr_parameters, n_jobs=-1)
gscv_lr_stem.fit(stem_train, twenty_train_full.target)

### Вывод полученных результатов анализа

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predicted_dt = gscv_dt.predict(twenty_test_full.data)
predicted_knn = gscv_knn.predict(twenty_test_full.data)
predicted_lr = gscv_lr.predict(twenty_test_full.data)

predicted_dt_stem = gscv_dt_stem.predict(twenty_test_full.data)
predicted_knn_stem = gscv_knn_stem.predict(twenty_test_full.data)
predicted_lr_stem = gscv_lr_stem.predict(twenty_test_full.data)


In [ ]:
print('Дерево решений (DT)\n')
print(classification_report(twenty_test_full.target, predicted_dt, target_names=categories))
print(gscv_dt.best_params_)

In [ ]:
print('Дерево решений (DT) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_dt_stem, target_names=categories))
print(gscv_dt_stem.best_params_)

In [ ]:
print('\nK-ближайших соседей (KNN)\n')
print(classification_report(twenty_test_full.target, predicted_knn, target_names=categories))
print(gscv_knn.best_params_)


In [ ]:
print('\nK-ближайших соседей (KNN) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_knn_stem, target_names=categories))
print(gscv_knn_stem.best_params_)

In [ ]:
print('\nЛогистическая регрессия (LR)\n')
print(classification_report(twenty_test_full.target, predicted_lr, target_names=categories))
print(gscv_lr.best_params_)

In [ ]:
print('\nЛогистическая регрессия (LR) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_lr_stem, target_names=categories))
print(gscv_lr_stem.best_params_)

### Сравнительная таблица

In [ ]:
import pandas as pd

In [ ]:
# Создаем Excel-файл и записываем в него результаты для новых методов
writer = pd.ExcelWriter('result_updated.xlsx', engine='openpyxl')

# Дерево решений (DT) без стемминга
df_dt = pd.DataFrame(classification_report(predicted_dt, twenty_test_full.target, output_dict=True))
df_dt.to_excel(writer, sheet_name='DT без стемминга')

# Дерево решений (DT) со стеммингом
df_dt_stem = pd.DataFrame(classification_report(predicted_dt_stem, twenty_test_full.target, output_dict=True))
df_dt_stem.to_excel(writer, sheet_name='DT со стеммингом')

# K-ближайших соседей (KNN) без стемминга
df_knn = pd.DataFrame(classification_report(predicted_knn, twenty_test_full.target, output_dict=True))
df_knn.to_excel(writer, sheet_name='KNN без стемминга')

# K-ближайших соседей (KNN) со стеммингом
df_knn_stem = pd.DataFrame(classification_report(predicted_knn_stem, twenty_test_full.target, output_dict=True))
df_knn_stem.to_excel(writer, sheet_name='KNN со стеммингом')

# Логистическая регрессия (LR) без стемминга
df_lr = pd.DataFrame(classification_report(predicted_lr, twenty_test_full.target, output_dict=True))
df_lr.to_excel(writer, sheet_name='LR без стемминга')

# Логистическая регрессия (LR) со стеммингом
df_lr_stem = pd.DataFrame(classification_report(predicted_lr_stem, twenty_test_full.target, output_dict=True))
df_lr_stem.to_excel(writer, sheet_name='LR со стеммингом')

# Закрываем запись в Excel-файл
writer.close()
